# Tweets abnehmen und Sentiment analyse anwenden

Die Tweets aus dem Producer werden eingelesen und als PArquet abgelegt.
Darauf wird via *textblob* eine Sentiment analyse gemacht und jedem Tweet ein Rating zugewiesen.

In [0]:
# !pip install textblob
# !pip install bs4

In [0]:
# Libraries einlesen

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *   # including udf
from pyspark.sql.types import *   
from pyspark.sql import functions as F
from textblob import TextBlob
from datetime import datetime
import random
import requests
import pandas as pd
import json
import time
from bs4 import BeautifulSoup 

In [0]:
# Definition der Textklassification via Textblob

@udf
def polarity_detection(text):
    return TextBlob(text).sentiment.polarity

@udf
def subjectivity_detection(text):
    return TextBlob(text).sentiment.subjectivity


In [0]:
# Funtion um Bitcoint Preis zu laden

@udf
def get_actual_crypto_price(crypto):
  base_url = 'https://coinmarketcap.com'
  request = requests.get(base_url)
  soup = BeautifulSoup(request.content, 'html.parser')
  data = soup.find('script', id="__NEXT_DATA__", type="application/json")
  coins = {}

  coin_data = json.loads(data.contents[0])
  listings = coin_data['props']['initialState']['cryptocurrency']['listingLatest']['data']

  for i in listings:
    crypto_curr = i['name']
    if crypto_curr.lower() == crypto.lower():
      evaluate_price = i['quotes'][2]
      #print(evaluate_price)
      coins[str(i['id'])] = i['slug']
      coins['currency'] = evaluate_price['name']
      coins['actual_price'] = evaluate_price['price']
      coins['percentChange24h'] = evaluate_price['percentChange24h']
  return coins['actual_price']


In [0]:
# Erstellen eines local/private StreamingContext (SparkContext 'sc' besteht in databricks bereits)

ssc = StreamingContext(sc, 2)   # batch interval = 2
stream = ssc.socketTextStream("localhost", 9997)

'''
# Ausgabe des stream in die Konsole für 3 Minuten, danach Abbruch
stream.pprint()
try:
  ssc.start()                             
  ssc.awaitTerminationOrTimeout(180)  # Ausgabe im consumer erst nach timeout möglich (sekunden)
finally:
  ssc.stop(False)
'''

Out[2]: '\n# Ausgabe des stream in die Konsole für 3 Minuten, danach Abbruch\nstream.pprint()\ntry:\n ssc.start() \n ssc.awaitTerminationOrTimeout(180) # Ausgabe im consumer erst nach timeout möglich (sekunden)\nfinally:\n ssc.stop(False)\n'

In [0]:
# Funktionen um Dstream in Dataframe mit 6-Sekunden-Fenster zu transformieren

'''
# das Problem ist bereits bei 'lines' wenn man das ausgibt sieht man das Durcheinander das da kommt...
# lines ist bereits ein Dataframe
# Der stream aus dem command oben ist dagegen ein RDD, das sieht noch ok aus. Allenfalls könnte man mit der .map() funktion auf dem stream splitten?
# vielleicht so etwas:

splittedStream = stream.map(split("t_end"))
lines = spark.createDataFrame(splittetStream)

# ich glaube wir brauchen etwas anderes als den COde wie wir ihn bisher hatten:

# Dstream abnehmen und in DF laden
lines = spark \
        .readStream.format("socket") \
        .option("host", "localhost") \
        .option("port", 9997) \
        .load()

'''

# Aufsplitten des Streams in TImestamp und Text
structuredStream = lines.select( \
                  split(lines.value, "_t_end_")[0].alias("timestamp") \
                , split(lines.value, "_t_end_")[1].alias("text") \
               )

# Timstamp formatieren
structuredStream = structuredStream.withColumn('timestamp', structuredStream.timestamp.astype('Timestamp'))

# Sentiment einfügen
structuredStream = structuredStream.withColumn("subjectivity", subjectivity_detection("text").cast('float'))
structuredStream = structuredStream.withColumn("polarity", polarity_detection("text").cast('float'))

# Währungskolonne Einfügen
structuredStream = structuredStream.withColumn("crypto", lit('bitcoin'))

# Erstellen eines 6 Sekunden-Fensters (als Basis für alle Analysen)
windowedStream = structuredStream \
        .withWatermark('timestamp', '10 seconds') \
        .groupBy(window("timestamp", "6 seconds", "6 seconds"))

# Aggregationsfunktion
aggregationsStream = windowedStream \
        .agg(count('text').alias("count_tweets") \
           , avg('subjectivity').alias('sub_avg') \
           , avg('polarity').alias('pol_avg') \
           , get_actual_crypto_price(first(col('crypto'))).alias('window_price') \
           , max('timestamp').alias('timestamp')
           )
'''

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2959400940596469> in <module> 
 22 '''
 23 
 ---> 24 structuredStream = stream.select( \
 25 split ( lines . value , "_t_end_" ) [ 0 ] . alias ( "timestamp" ) \ 
 26 , split ( lines . value , "_t_end_" ) [ 1 ] . alias ( "text" ) \ 

 AttributeError : 'DStream' object has no attribute 'select'

In [0]:
# Anzeige des aggregierten Streams 
# display(aggregationsStream)
# display(structuredStream)
display(lines)

ab hier funktioniert es nicht mehr:
-> der writestream will nicht starten: Watermark fehlt / oder dann writestream kennt kein watermark

In [0]:
# Sink der Daten in ein Parquet file
# dieser SCH.. kommt nicht zum laufen...
# watermark muss definiert werden - aber wie und wo?

query = aggregationsStream \
    .writeStream \
    .queryName("bc_table") \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "dbfs:/FileStore/bd_project") \
    .option("checkpointLocation", "./check") \
    .trigger(processingTime='30 seconds') \
    .start()

query.awaitTermination()


# Ab hier kein aktiver Code mehr

Zusätzliche Funktionen / variationen von Code / file handling / etc...

In [0]:
'''
# Funktion mit erweitertem preprocessing 

@udf
def preprocessing(lines):
    words = lines.select(explode(split(lines.value, " _t_end_ ")).alias("word"))
    words = words.na.replace('', None)
    words = words.na.drop()
    words = words.withColumn('word', F.regexp_replace('word', r'http\S+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '@\w+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '#', ''))
    words = words.withColumn('word', F.regexp_replace('word', 'RT', ''))
    words = words.withColumn('word', F.regexp_replace('word', ':', ''))
    return words
'''

File System functions

In [0]:
# Diretory (Kommentar entfernen damit es funktioniert)
%fs ls dbfs:/FileStore/

In [0]:
# Files und Folders rekursiv löschen (Kommentar entfernen damit es funktioniert)
%fs rm -r dbfs:/FileStore/import-stage/

In [0]:
# Folder erstellen
# dbutils.fs.mkdirs("dbfs/FileStore/bd_project/test")

In [0]:
# Directory anzeigen
# dbutils.fs.ls("dbfs:/dbfs")